In [ ]:
# 1. Import necessary libraries
import os
from google.colab import drive
from roboflow import Roboflow
import configparser

In [ ]:
# 2. Set up environment
!nvidia-smi
drive.mount('/content/drive')
HOME = os.getcwd()
print(f"Working directory: {HOME}")

Sun Sep 22 20:31:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# 3. Install required packages
!pip install -q roboflow ultralytics

In [ ]:
# 4. Download and prepare dataset
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# Read configuration
config = configparser.ConfigParser()
config.read('config.ini')

rf = Roboflow(api_key=config['Roboflow']['API_KEY'])
project = rf.workspace(config['Project']['WORKSPACE']).project(config['Project']['PROJECT_NAME'])
version = project.version(config['Project']['VERSION'])
dataset = version.download("yolov8")

/content


In [ ]:
# Update data.yaml file
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml


In [ ]:
# 5. Train YOLOv8 model
%cd {HOME}

!yolo task=detect \
mode=train \
model=yolov8x.pt \
data={dataset.location}/data.yaml \
batch=4 \
epochs=50 \
imgsz=1280 \
plots=True

In [ ]:
# 6. Validate the trained model
!yolo task=detect \
mode=val \
model={HOME}/runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml

In [ ]:
# 7. Deploy model on Roboflow
project.version(dataset.version).deploy(model_type='yolov8', model_path=f"{HOME}/runs/detect/train")